In [1]:
%env CUDA_LAUNCH_BLOCKING=0
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_LAUNCH_BLOCKING=0
env: CUDA_VISIBLE_DEVICES=0


Install Real-ESRGAN from [repository](https://github.com/xinntao/Real-ESRGAN.git)

Perform super-resolution for the training set with synthetic data using a similar command
```
for f in ANGRER DISGUST FEAR HAPPINESS SADNESS SURPRISE; do echo $f; CUDA_VISIBLE_DEVICES=0 python3 inference_realesrgan.py -n RealESRGAN_x4plus -i /home/HDD6TB/datasets/emotions/ABAW/eccv_4/training_set_synthetic_images/$f --face_enhance -s 2 -o /home/HDD6TB/datasets/emotions/ABAW/eccv_4/SR/training_set_synthetic_images/$f; done;
```

In [2]:
# Importing data

# !upload kaggle.json to use API for downloading abaw dataset from private repo!
from google.colab import files
#files.upload()
!echo '{"username":"dmitrykozlov10","key":"c386727d69ef70f1a72d4eb0b59d52b1"}' > kaggle.json

# move kaggle key to .kaggle 
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!pip install -q kaggle

!kaggle datasets download -d dmitrykozlov10/abaw-lsd --unzip



100% 828M/829M [00:21<00:00, 43.8MB/s]
100% 829M/829M [00:21<00:00, 40.1MB/s]


In [3]:
# mark folders

train_dir='training_set_synthetic_images/training_set_synthetic_images'
test_dir='validation_set_real_images/validation_set_real_images'

In [4]:
#!pip install mord

import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,roc_auc_score,average_precision_score
#import mord

from collections import defaultdict
import os
import glob
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import random
import csv  
import matplotlib.pyplot as plt

In [5]:
def one_hot_transfer(label,class_num):
    return np.eye(class_num)[label]

def metric_for_Exp(gt,pred,class_num=6):
    # compute_acc
    acc = accuracy_score(gt,pred)
    # compute_F1
    gt = one_hot_transfer(gt,class_num).detach().cpu().numpy()
    pred = one_hot_transfer(pred,class_num).detach().cpu().numpy()
    F1 = []
    for i in range(class_num):
        gt_ = gt[:,i]
        pred_ = pred[:,i]
        F1.append(f1_score(gt_.flatten(), pred_))
    F1_mean = np.mean(F1)
    return F1_mean,acc,F1


# fix seed
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)

seed_everything(1337)

# Train

In [6]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

!pip install pytorch_lightning lightning --quiet
import lightning.pytorch as pl


print(f"Torch: {torch.__version__}")

Torch: 2.0.0+cu118


In [7]:
# Training settings
batch_size = 64 #48# 96# 32 #40 #32 #16 #8 #
epochs = 8
lr = 3e-5
gamma = 0.7
seed = 42
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

device = 'cuda' if use_cuda else 'cpu'
print (device)

True
cuda


In [8]:
IMG_SIZE = 224

train_transforms = transforms.Compose(
    [
        #transforms.RandomRotation(degrees=30),
        #transforms.RandomResizedCrop(IMG_SIZE),
        #transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

In [9]:
kwargs = {'num_workers': 2, 'pin_memory': False} if use_cuda else {}

print(train_dir,test_dir)
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)

(unique, counts) = np.unique(train_dataset.targets, return_counts=True)
cw=1/counts
#cw/=cw.min()
cw*=counts.mean()
class_weights = {i:cwi for i,cwi in zip(unique,cw)}
array_weights = np.array(list(class_weights.values()), dtype=float)
tensor_weights = torch.from_numpy(array_weights)
print(counts, class_weights.values(), tensor_weights)

#split to train val
train_size = int(0.75 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

print('split to train\val', len(train_dataset), len(val_dataset))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, **kwargs)



test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transforms)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs) 

print(len(train_dataset), len(test_dataset))

training_set_synthetic_images/training_set_synthetic_images validation_set_real_images/validation_set_real_images
[ 18286  15150  10923  73285 144631  14976] dict_values([2.5269878595646946, 3.0500660066006597, 4.230385425249474, 0.6305314866616634, 0.31949236332459846, 3.0855034722222223]) tensor([2.5270, 3.0501, 4.2304, 0.6305, 0.3195, 3.0855], dtype=torch.float64)
split to trainal 207938 69313
207938 4670


In [10]:
class_to_idx=test_dataset.class_to_idx
print(class_to_idx)
idx_to_class={idx:cls for cls,idx in class_to_idx.items()}
print(idx_to_class)

{'ANGRER': 0, 'DISGUST': 1, 'FEAR': 2, 'HAPPINESS': 3, 'SADNESS': 4, 'SURPRISE': 5}
{0: 'ANGRER', 1: 'DISGUST', 2: 'FEAR', 3: 'HAPPINESS', 4: 'SADNESS', 5: 'SURPRISE'}


#Importing pretrained model

In [33]:
from torchvision.models import resnet18
from torchvision.models import efficientnet_b0
from torchmetrics.functional import accuracy
from torchmetrics.classification import MulticlassF1Score
from lightning import LightningModule

#model = resnet18(pretrained=True)


class lsdResnet(LightningModule):
  def __init__(self, pretrained: bool = True, weights = None) -> None:
        super().__init__()
        self.pretrained = pretrained
        self.weights = weights

        if pretrained:
            # <YOUR CODE HERE>
            #self.model = resnet18(pretrained=True)
            self.model = efficientnet_b0(pretrained=True)

            self.model.fc = nn.Identity()
            self.classifier = nn.Sequential(
                nn.BatchNorm1d(1000),
                nn.LeakyReLU(),
                nn.Linear(1000, 256), 
                nn.LeakyReLU(),
                nn.Linear(256, 64),
                nn.Dropout(p=0.3),
                nn.LeakyReLU(),
                nn.Linear(64, 6)
            )
            #self.optimizer = torch.optim.Adam(self.classifier.parameters())
        else:
            # <YOUR CODE HERE>
            self.model = resnet18(pretrained=False)
            #self.optimizer = torch.optim.Adam(self.model.parameters())

        self.loss = nn.CrossEntropyLoss(weight=weights)

  def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.classifier.parameters(), lr=3e-3)
        sch = torch.optim.lr_scheduler.StepLR(
        optimizer, step_size  = 10 , gamma = 0.5)
 #learning rate scheduler
        return {
            "optimizer":optimizer,
            "lr_scheduler" : {
                "scheduler" : sch,
                "monitor" : "train_loss",
                
            }
        }

  def forward(self, x) -> torch.Tensor:
        if self.pretrained:
            # <YOUR CODE HERE>
            with torch.no_grad():
                features = self.model(x)
            preds = self.classifier(features)
        else:
            # <YOUR CODE HERE>
            preds = self.model(x)
        return preds

  #def configure_optimizers(self):
        #return self.optimizer

  def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        images, target = train_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        self.log("train_loss", loss, prog_bar=True)
        #self.log("F1Average", MulticlassF1Score(num_classes=6, average='micro').to(device))
        #print("F1Average", F1Average(preds, target))
        return loss

  def validation_step(self, val_batch, batch_idx) -> None:
        images, target = val_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        #accuracy(task="multiclass", num_classes=6)
        F1Average = MulticlassF1Score(num_classes=6, average='macro').to(device)
        F1Average(preds, target)
        self.log("val_loss", loss, prog_bar=True)
        #self.log("F1Average", F1Average, prog_bar=True)
        #self.log("F1Average", MulticlassF1Score(num_classes=6, average='micro').to(device))

        #print("F1Average on val", F1Average(preds, target))

  def test_step(self, test_batch, batch_idx) -> None:
        images, target = test_batch
        preds = self.forward(images)
        loss = self.loss(preds, target)
        #accuracy(task="multiclass", num_classes=6)
        F1Average = MulticlassF1Score(num_classes=6, average='macro').to(device)
        
        self.log("test_loss", loss, prog_bar=True)
        self.log("F1Average", F1Average(preds, target), prog_bar=True)
        print("F1Average on test", F1Average(preds, target))

In [34]:
lsdModel = lsdResnet(pretrained=True, weights = tensor_weights.float())

trainer = pl.Trainer(accelerator = "gpu", max_epochs = 2)
trainer.fit(lsdModel, train_loader, val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name       | Type             | Params
------------------------------------------------
0 | model      | EfficientNet     | 5.3 M 
1 | classifier | Sequential       | 275 K 
2 | loss       | CrossEntropyLoss | 0     
------------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.255    T

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=2` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


In [35]:
trainer.test(lsdModel, test_loader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

F1Average on test tensor(0.2969, device='cuda:0')
F1Average on test tensor(0.7344, device='cuda:0')
F1Average on test tensor(0.7969, device='cuda:0')
F1Average on test tensor(0.6875, device='cuda:0')
F1Average on test tensor(0.4688, device='cuda:0')
F1Average on test tensor(0.4531, device='cuda:0')
F1Average on test tensor(0.9062, device='cuda:0')
F1Average on test tensor(0.7656, device='cuda:0')
F1Average on test tensor(0.4844, device='cuda:0')
F1Average on test tensor(0.7031, device='cuda:0')
F1Average on test tensor(0.9531, device='cuda:0')
F1Average on test tensor(0.8750, device='cuda:0')
F1Average on test tensor(0.6719, device='cuda:0')
F1Average on test tensor(0.9688, device='cuda:0')
F1Average on test tensor(0.9531, device='cuda:0')
F1Average on test tensor(0.7344, device='cuda:0')
F1Average on test tensor(0.5781, device='cuda:0')
F1Average on test tensor(0.4531, device='cuda:0')
F1Average on test tensor(0.8750, device='cuda:0')
F1Average on test tensor(0.9375, device='cuda:0')


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         F1Average         │    0.6683083772659302     │
│         test_loss         │    1.1414176225662231     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.1414176225662231, 'F1Average': 0.6683083772659302}]

In [ ]:
model_scripted = torch.jit.script(lsdModel) # Export to TorchScript
model_scripted.save('/content/lsdModel5Epochs.pt')





In [ ]:
!pwd

/content


# Testing model on 1 picture


In [ ]:
print(idx_to_class.get(1))

DISGUST


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#upload a .pt model

#files.upload()
#files.upload()
idx_to_class = {0: 'ANGRER', 1: 'DISGUST', 2: 'FEAR', 3: 'HAPPINESS', 4: 'SADNESS', 5: 'SURPRISE'}

image = Image.open("/content/drive/MyDrive/photo1.jpg")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image_tensor = preprocess(image).unsqueeze(0)

modelTest = torch.load("/content/FER_trained_model.pt")

modelTest.eval()

output = modelTest(image_tensor)
print(idx_to_class.get(output.data.cpu().numpy().argmax()))

AttributeError: ignored

In [ ]:


def test_accuracy(net, testloader):
    correct = 0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)

            # the class with the highest energy is what we choose as prediction
            predicted = torch.max(outputs.data, 1)[1]

            correct += (predicted == labels).sum().item()
    
    return correct / len(testloader.dataset)
    
#modelTest.to(device)
def test_accuracy_per_class(net, testloader, device):
    trainset = testloader.dataset
    correct_pred = {classname: 0 for classname in trainset.classes}
    total_pred = {classname: 0 for classname in trainset.classes}

    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            outputs = net(images).to(device)
            predicted = torch.max(outputs.data, 1)[1]

            # collect the correct predictions for each class
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    correct_pred[trainset.classes[label]] += 1
                total_pred[trainset.classes[label]] += 1
    
    accuracy_per_class = {classname: 0 for classname in trainset.classes}
    for classname, correct_count in correct_pred.items():
        accuracy = (100 * float(correct_count)) / total_pred[classname]
        accuracy_per_class[classname] = accuracy

    return accuracy_per_class



print(
    'Overall accuracy of the network  '
    f'{(overall_accuracy * 100):.2f} %\n'
    'on the 10000 test images'
)

accuracy_per_class = test_accuracy_per_class(lsdModel, test_loader, device)

print('Accuracy per class\n')
for classname, accuracy in accuracy_per_class.items():
    print(f'{classname:12s} {accuracy:.2f} %')

AttributeError: ignored

Load fer model

In [ ]:
def test_accuracy_per_class(net, testloader, device):
    trainset = testloader.dataset
    correct_pred = {classname: 0 for classname in trainset.classes}
    total_pred = {classname: 0 for classname in trainset.classes}

    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            outputs = net(images).to(device)
            predicted = torch.max(outputs.data, 1)[1]

            # collect the correct predictions for each class
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    correct_pred[trainset.classes[label]] += 1
                total_pred[trainset.classes[label]] += 1
    
    accuracy_per_class = {classname: 0 for classname in trainset.classes}
    for classname, correct_count in correct_pred.items():
        accuracy = (100 * float(correct_count)) / total_pred[classname]
        accuracy_per_class[classname] = accuracy

    print(
    'Overall accuracy of the network  '
    f'{(accuracy_per_class * 100):.2f} %\n'
    'on the 10000 test images'
)

    return accuracy_per_class

def test_accuracy(net, testloader):
    correct = 0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)

            # the class with the highest energy is what we choose as prediction
            predicted = torch.max(outputs.data, 1)[1]

            correct += (predicted == labels).sum().item()
    
    return correct / len(testloader.dataset)